In [ ]:
from transformers import BertModel,BertTokenizer
import torch
from torch.utils.data import Dataset,DataLoader
from datasets import load_from_disk

In [ ]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768,2)
    # 前向推理运算
    def forward(self,input_ids,attention_mask,token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,attention_mask= attention_mask,token_type_ids=token_type_ids)
        
        out= self.fc(out.last_hidden_state[:,0])
        out = out.softmax(dim=1)
        return out

In [ ]:
DEVICE= torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_name="bert-base-chinese"
model_name=r"D:\Workspace\regression\huggingface\model\bert-base-chinese\models--bert-base-chinese\snapshots\c30a6ed22ab4564dc1e3b2ecbf6e766b0611a33f"
tokenizer = BertTokenizer.from_pretrained(model_name)
pretrained = BertModel.from_pretrained(model_name).to(DEVICE)
names=["负向评价","正向评价"]

print(DEVICE)
model=Model().to(DEVICE)

In [ ]:
def collate_fn(data):
    sentes = []
    sentes.append(data)
    
    data = tokenizer.batch_encode_plus(
        batch_text_or_text_pairs=sentes,
        padding="max_length",  # 填充到最大长度
        truncation=True,       # 截断超过最大长度的序列
        max_length=30,         # 最大长度为 10
        return_length=True,
        return_tensors="pt",
    )
    input_ids = data["input_ids"]
    attention_mask = data["attention_mask"]
    token_type_ids = data["token_type_ids"]
    
    return input_ids,attention_mask,token_type_ids

In [ ]:
def test(data):
    model.load_state_dict(torch.load("params/2bert.pt"))
    model.eval()
    
    input_ids,attention_mask,token_type_ids = collate_fn(data)
    
    input_ids,attention_mask,token_type_ids =  input_ids.to(DEVICE),attention_mask.to(DEVICE),token_type_ids.to(DEVICE)

    with torch.no_grad():
        out = model(input_ids=input_ids,attention_mask= attention_mask,token_type_ids=token_type_ids)
        out = out.softmax(dim=1)
        print（"模型判定："names[out],"\n"）
        
if __name__=='__main__':
    test("酒店不好住啊")